In [100]:
import pandas as pd
df=pd.read_csv("netflix_titles.csv")

In [101]:
df.shape, df.columns.tolist()

((8807, 12),
 ['show_id',
  'type',
  'title',
  'director',
  'cast',
  'country',
  'date_added',
  'release_year',
  'rating',
  'duration',
  'listed_in',
  'description'])

In [102]:
# 1. Initial inspection
df.head() 
df.info()
df.describe(include='all').T
df.isnull().sum().sort_values(ascending =  False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


director        2634
country          831
cast             825
date_added        10
rating             4
duration           3
show_id            0
type               0
title              0
release_year       0
listed_in          0
description        0
dtype: int64

In [103]:
 # 2. Clean column names (uniform)
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(' ', '_')
              .str.replace(r'[^\w_]', '', regex=True)
)


In [104]:
# 3. Remove exact duplicate rows (if any)
before = len(df)
df = df.drop_duplicates()
after = len(df)
print("Dropped duplicates:", before-after)


Dropped duplicates: 0


In [105]:
# 4. Handle missing values (strategy per column)
# - director, cast, country, rating, description: fill with 'Unknown' 
fill_with_unknown = ['director', 'cast', 'country', 'rating', 'description']
for c in fill_with_unknown:
    if c in df.columns:
        df[c] = df[c].fillna('Unknown')


In [106]:
# 5. Standardize text fields (strip whitespace, normalize case where useful)
text_cols = ['type','title','director','cast','country','rating','listed_in','description']
for c in text_cols:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()
        

In [107]:
# 6. Fix date formats: date_added -> datetime
if 'date_added' in df.columns:
    df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')


In [108]:
# 7. Convert release_year to integer
if 'release_year' in df.columns:
    df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce').astype('Int64')


In [109]:
# 8. Parse duration into numeric + unit
# Example values: "90 min" or "1 Season"
if 'duration' in df.columns:
    # separate number and unit
    dur = df['duration'].str.extract(r'(?P<duration_num>\d+)\s*(?P<duration_unit>\w+)', expand=True)
    df['duration_num']  = pd.to_numeric(dur['duration_num'], errors='coerce').astype('Int64')
    df['duration_unit'] = dur['duration_unit'].str.lower().fillna('unknown')


In [110]:
# 9. Standardize rating (if you want categories)
if 'rating' in df.columns:
    df['rating'] = df['rating'].replace(['UR'], 'Unknown')   # example
    df['rating'] = df['rating'].replace('', 'Unknown')


In [111]:
# 10. Normalize country and cast lists
# Many entries are comma-separated lists — trim whitespace
if 'country' in df.columns:
    df['country'] = df['country'].replace('Unknown', pd.NA) 
    df['primary_country'] = df['country'].fillna('').str.split(',').str[0].str.strip().replace('', pd.NA)


In [112]:
# 11. Remove rows missing critical info 
df = df[df['type'].notna()]


In [113]:
# 12. Type conversions & final checks
df.info()
df.isnull().sum().sort_values(ascending=False).head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   show_id          8807 non-null   object        
 1   type             8807 non-null   object        
 2   title            8807 non-null   object        
 3   director         8807 non-null   object        
 4   cast             8807 non-null   object        
 5   country          7976 non-null   object        
 6   date_added       8709 non-null   datetime64[ns]
 7   release_year     8807 non-null   Int64         
 8   rating           8807 non-null   object        
 9   duration         8804 non-null   object        
 10  listed_in        8807 non-null   object        
 11  description      8807 non-null   object        
 12  duration_num     8804 non-null   Int64         
 13  duration_unit    8807 non-null   object        
 14  primary_country  7974 non-null   object 

primary_country    833
country            831
date_added          98
duration             3
duration_num         3
show_id              0
type                 0
title                0
director             0
cast                 0
release_year         0
rating               0
listed_in            0
description          0
duration_unit        0
dtype: int64

In [114]:
# 13. Saving cleaned dataset
df.to_csv("netflix_titles_cleaned.csv", index=False)
